In [ ]:
# Cell 1: Initial Setup - Local Runtime

import os

# For local runtime - use your actual computer path
# NOT Google Drive mounting

# Create project directory on your local machine
project_dir = os.path.expanduser('~/FinancialAI')  # This creates folder in your home directory
os.makedirs(project_dir, exist_ok=True)

print(f"✅ Project directory: {project_dir}")
print("\n💡 Files will be saved to your local computer")
print("💡 Next: Set your OpenAI API key in Cell 2")

✅ Project directory: /root/FinancialAI

💡 Files will be saved to your local computer
💡 Next: Set your OpenAI API key in Cell 2


In [ ]:
# Cell 2: Install packages and set API key (COMPLETE FIX)

print("📦 Installing required packages...")

# Install specific compatible versions
!pip install -q openai==1.3.0 datasets==2.14.0

print("✅ Packages installed!")

# Import and set API key
import os

# REPLACE WITH YOUR ACTUAL API KEY
os.environ['OPENAI_API_KEY'] = ''

print("\n✅ API key set!")

# Test API key with simple import (no client creation yet)
try:
    from openai import OpenAI
    print("✅ OpenAI library imported successfully!")
    print("🎉 Ready to proceed!")

except Exception as e:
    print(f"❌ Error: {e}")

📦 Installing required packages...
✅ Packages installed!

✅ API key set!
✅ OpenAI library imported successfully!
🎉 Ready to proceed!


In [ ]:
# Cell 2.5: Fix PyArrow compatibility issue

print("🔧 Fixing PyArrow compatibility...")

# Uninstall and reinstall compatible versions
!pip uninstall -y pyarrow datasets -q
!pip install -q pyarrow==14.0.1 datasets==2.14.0

print("✅ Fixed!")
print("⚠️ IMPORTANT: Restart the kernel now!")
print("\nSteps:")
print("1. Click 'Kernel' in the menu")
print("2. Click 'Restart Kernel'")
print("3. After restart, run Cell 1 and Cell 2 again")
print("4. Then run Cell 3")

🔧 Fixing PyArrow compatibility...
✅ Fixed!
⚠️ IMPORTANT: Restart the kernel now!

Steps:
1. Click 'Kernel' in the menu
2. Click 'Restart Kernel'
3. After restart, run Cell 1 and Cell 2 again
4. Then run Cell 3


In [ ]:
# Cell 3: Download FinQA Manually (Complete Solution)

import requests
import json
import os

print("📚 Downloading FinQA dataset manually...")
print("="*70)

# Project directory
project_dir = os.path.expanduser('~/FinancialAI')

# FinQA GitHub raw URLs
urls = {
    'train': 'https://raw.githubusercontent.com/czyssrs/FinQA/master/dataset/train.json',
    'validation': 'https://raw.githubusercontent.com/czyssrs/FinQA/master/dataset/dev.json',
    'test': 'https://raw.githubusercontent.com/czyssrs/FinQA/master/dataset/test.json'
}

dataset = {}

for split, url in urls.items():
    print(f"\n📥 Downloading {split} data...")

    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()

        data = response.json()
        dataset[split] = data

        print(f"✅ {split}: {len(data):,} examples")

    except Exception as e:
        print(f"❌ Error downloading {split}: {e}")
        dataset[split] = []

# Save locally
for split, data in dataset.items():
    filepath = os.path.join(project_dir, f'finqa_{split}.json')
    with open(filepath, 'w') as f:
        json.dump(data, f)
    print(f"💾 Saved {split} to: {filepath}")

print(f"\n✅ Dataset downloaded successfully!")
print(f"📊 Training examples: {len(dataset['train']):,}")
print(f"📊 Validation examples: {len(dataset['validation']):,}")
print(f"📊 Test examples: {len(dataset['test']):,}")

# Inspect a sample
print("\n" + "="*70)
print("📄 SAMPLE EXAMPLE:")
print("="*70)

if len(dataset['train']) > 0:
    sample = dataset['train'][0]

    print(f"\n🔹 Pre-text: {sample.get('pre_text', 'N/A')[:200]}...")
    print(f"\n🔹 Question: {sample.get('question', 'N/A')}")
    print(f"\n🔹 Answer: {sample.get('answer', 'N/A')}")
else:
    print("⚠️ No training data loaded")

print("\n✅ Dataset ready for formatting!")

📚 Downloading FinQA dataset manually...

📥 Downloading train data...
✅ train: 6,251 examples

📥 Downloading validation data...
✅ validation: 883 examples

📥 Downloading test data...
✅ test: 1,147 examples
💾 Saved train to: /root/FinancialAI/finqa_train.json
💾 Saved validation to: /root/FinancialAI/finqa_validation.json
💾 Saved test to: /root/FinancialAI/finqa_test.json

✅ Dataset downloaded successfully!
📊 Training examples: 6,251
📊 Validation examples: 883
📊 Test examples: 1,147

📄 SAMPLE EXAMPLE:

🔹 Pre-text: ['interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .', 'if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million .', 'foreign currency exposure as more fully described in note 2i .', 'in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s .', 'dollar-based exposures by

In [ ]:
# COMPLETE FIX: Run this as Cell 4 (Replace old Cell 4)

import json

def is_safe_example(example):
    """Filter unsafe examples"""
    # Access question and answer from the 'qa' sub-dictionary
    qa = example.get('qa', {})
    question = str(qa.get('question', '')).lower()
    answer = str(qa.get('answer', '')).lower()

    # Filter out examples with missing question or answer
    if not question or question == 'n/a' or not answer or answer == 'n/a':
        return False

    # Filter unsafe keywords
    unsafe = ['fraud', 'illegal', 'insider', 'should i invest', 'recommend buying']
    if any(word in question + answer for word in unsafe):
        return False

    # Keep only factual questions (can be adjusted based on desired model behavior)
    # For FinQA, most questions are factual, so a less strict filter might be fine.
    # Let's be a bit more permissive, focusing on safety first.
    # factual = ['what was', 'what is', 'how much', 'calculate', 'percentage']
    # return any(word in question for word in factual)
    return True # Allow all non-unsafe, non-empty questions for now


def format_finqa_safe(example):
    """Format with safety"""

    context = ""
    if example.get('pre_text'):
        context += '\n'.join(example['pre_text']) + "\n\n"
    if example.get('table'):
        # Format table data more readably if it's a list of lists
        table_str = ""
        for row in example['table']:
            table_str += '| ' + ' | '.join(map(str, row)) + ' |\n'
        context += "Table:\n" + table_str + "\n\n"
    if example.get('post_text'):
        context += '\n'.join(example['post_text'])

    # Access question and answer from the 'qa' sub-dictionary
    qa = example.get('qa', {})
    question = str(qa.get('question', ''))
    answer = str(qa.get('answer', ''))

    if not question or question == 'N/A' or not answer or answer == 'N/A':
        return None

    return {
        "messages": [
            {
                "role": "system",
                "content": "You are a financial analyst. Provide FACTUAL analysis only based on provided data. DO NOT give investment advice."
            },
            {
                "role": "user",
                "content": f"Context:\n{context}\n\nQuestion: {question}"
            },
            {
                "role": "assistant",
                "content": answer
            }
        ]
    }


# Filter and format
print("🔄 Filtering and formatting safe data...")

training_data = []
for example in dataset['train'][:1000]: # Limiting to 1000 examples for faster processing
    if is_safe_example(example):
        formatted = format_finqa_safe(example)
        if formatted:
            training_data.append(formatted)

print(f"✅ Safe training examples: {len(training_data)}")

validation_data = []
for example in dataset['validation'][:100]: # Limiting to 100 examples
    if is_safe_example(example):
        formatted = format_finqa_safe(example)
        if formatted:
            validation_data.append(formatted)

print(f"✅ Safe validation examples: {len(validation_data)}")

🔄 Filtering and formatting safe data...
✅ Safe training examples: 984
✅ Safe validation examples: 100


In [ ]:
# Cell 5: Save formatted data as JSONL files (Local)

import json
import os

# Use local project directory
project_dir = os.path.expanduser('~/FinancialAI')

# File paths
train_file = os.path.join(project_dir, 'finqa_train.jsonl')
val_file = os.path.join(project_dir, 'finqa_validation.jsonl')

print("💾 Saving formatted data...")
print("="*70)

# Save training data
print(f"\n📝 Saving training data to: {train_file}")
with open(train_file, 'w') as f:
    for item in training_data:
        f.write(json.dumps(item) + '\n')

print(f"✅ Saved {len(training_data)} training examples")

# Save validation data
print(f"\n📝 Saving validation data to: {val_file}")
with open(val_file, 'w') as f:
    for item in validation_data:
        f.write(json.dumps(item) + '\n')

print(f"✅ Saved {len(validation_data)} validation examples")

# Check file sizes
train_size = os.path.getsize(train_file) / 1024 / 1024  # MB
val_size = os.path.getsize(val_file) / 1024 / 1024

print(f"\n📊 File sizes:")
print(f"   Training: {train_size:.2f} MB")
print(f"   Validation: {val_size:.2f} MB")

print(f"\n📁 Files saved to: {project_dir}")
print("\n✅ Data saved successfully!")

💾 Saving formatted data...

📝 Saving training data to: /root/FinancialAI/finqa_train.jsonl
✅ Saved 984 training examples

📝 Saving validation data to: /root/FinancialAI/finqa_validation.jsonl
✅ Saved 100 validation examples

📊 File sizes:
   Training: 4.09 MB
   Validation: 0.42 MB

📁 Files saved to: /root/FinancialAI

✅ Data saved successfully!


In [ ]:
# Cell 2B: Fix OpenAI and httpx versions

print("🔧 Fixing package versions...")

!pip uninstall -y openai httpx
!pip install -q openai==1.3.0 httpx==0.24.1

print("✅ Fixed!")
print("⚠️ RESTART KERNEL NOW!")
print("\nAfter restart:")
print("1. Run Cell 1 (setup)")
print("2. Run Cell 2 (API key)")
print("3. Continue from Cell 3")

🔧 Fixing package versions...
Found existing installation: openai 1.3.0
Uninstalling openai-1.3.0:
  Successfully uninstalled openai-1.3.0
Found existing installation: httpx 0.24.1
Uninstalling httpx-0.24.1:
  Successfully uninstalled httpx-0.24.1
✅ Fixed!
⚠️ RESTART KERNEL NOW!

After restart:
1. Run Cell 1 (setup)
2. Run Cell 2 (API key)
3. Continue from Cell 3


In [ ]:
# Cell 6: Upload to OpenAI (Error Handling Version)

import os
import json

# Local project directory
project_dir = os.path.expanduser('~/FinancialAI')
train_file = os.path.join(project_dir, 'finqa_train.jsonl')
val_file = os.path.join(project_dir, 'finqa_validation.jsonl')

print("📤 Uploading training data to OpenAI...")
print("="*70)

# Try creating client with error handling
try:
    from openai import OpenAI
    client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
except TypeError:
    # Fallback for version issues
    print("⚠️ Using fallback client initialization...")
    import openai
    openai.api_key = os.environ['OPENAI_API_KEY']
    from openai import OpenAI
    client = OpenAI()

# Upload training file
print("\n📤 Uploading training file...")
print(f"   File: {train_file}")

with open(train_file, 'rb') as f:
    train_upload = client.files.create(
        file=f,
        purpose='fine-tune'
    )

train_file_id = train_upload.id
print(f"✅ Training file uploaded!")
print(f"   File ID: {train_file_id}")

# Upload validation file
print("\n📤 Uploading validation file...")
print(f"   File: {val_file}")

with open(val_file, 'rb') as f:
    val_upload = client.files.create(
        file=f,
        purpose='fine-tune'
    )

val_file_id = val_upload.id
print(f"✅ Validation file uploaded!")
print(f"   File ID: {val_file_id}")

# Save file IDs
file_ids = {
    'train_file_id': train_file_id,
    'val_file_id': val_file_id
}

file_ids_path = os.path.join(project_dir, 'openai_file_ids.json')
with open(file_ids_path, 'w') as f:
    json.dump(file_ids, f, indent=2)

print(f"\n💾 File IDs saved to: {file_ids_path}")
print("\n✅ Upload complete! Ready for fine-tuning!")

📤 Uploading training data to OpenAI...

📤 Uploading training file...
   File: /root/FinancialAI/finqa_train.jsonl
✅ Training file uploaded!
   File ID: file-AA35b78oGC4CYJrkLNv2uk

📤 Uploading validation file...
   File: /root/FinancialAI/finqa_validation.jsonl
✅ Validation file uploaded!
   File ID: file-F8AhNTR4561dG47nBdAycq

💾 File IDs saved to: /root/FinancialAI/openai_file_ids.json

✅ Upload complete! Ready for fine-tuning!


In [ ]:
# Cell 7: Fine-Tune GPT-4o-mini (Safer Model)

from openai import OpenAI
import json
import os

project_dir = os.path.expanduser('~/FinancialAI')

# Load file IDs
with open(os.path.join(project_dir, 'openai_file_ids.json'), 'r') as f:
    file_ids = json.load(f)

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

print("🚀 Starting GPT-4o-mini Fine-Tuning...")
print("="*70)
print("⚠️  This is MORE EXPENSIVE but SAFER")
print("💰 Expected cost: ~$50-60")
print()

# Confirm before proceeding
confirm = input("Type 'yes' to proceed with GPT-4o-mini fine-tuning: ")

if confirm.lower() != 'yes':
    print("❌ Cancelled")
else:
    # Create fine-tuning job with GPT-4o-mini
    fine_tune_job = client.fine_tuning.jobs.create(
        training_file=file_ids['train_file_id'],
        validation_file=file_ids['val_file_id'],
        model="gpt-4o-2024-08-06",  # ← Newer, safer model
        hyperparameters={
            "n_epochs": 3
        },
        suffix="finqa-financial"
    )

    job_id = fine_tune_job.id

    print(f"\n✅ Fine-tuning job started!")
    print(f"📋 Job ID: {job_id}")
    print(f"📊 Model: gpt-4o-mini-2024-07-18")

    # Save job info
    job_info = {
        'job_id': job_id,
        'model': 'gpt-4o-mini-2024-07-18',
        'training_file': file_ids['train_file_id'],
        'validation_file': file_ids['val_file_id'],
        'n_epochs': 3,
        'num_examples': len(training_data) if 'training_data' in dir() else 1000
    }

    with open(os.path.join(project_dir, 'finetuning_job_info.json'), 'w') as f:
        json.dump(job_info, f, indent=2)

    print(f"\n⏳ Run Cell 8 to monitor progress")

🚀 Starting GPT-4o-mini Fine-Tuning...
⚠️  This is MORE EXPENSIVE but SAFER
💰 Expected cost: ~$50-60

Type 'yes' to proceed with GPT-4o-mini fine-tuning: yes

✅ Fine-tuning job started!
📋 Job ID: ftjob-B6muR1Rh2BEe5JWIs5W9O3Jv
📊 Model: gpt-4o-mini-2024-07-18

⏳ Run Cell 8 to monitor progress


In [ ]:
# Cell 8: Monitor Fine-Tuning Status (Local Runtime)

from openai import OpenAI
import json
import os
import time
from datetime import datetime

# Local project directory
project_dir = os.path.expanduser('~/FinancialAI')

# Load job ID
with open(os.path.join(project_dir, 'finetuning_job_info.json'), 'r') as f:
    job_info = json.load(f)

job_id = job_info['job_id']

# Create OpenAI client
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

print("🔄 Monitoring Fine-Tuning Progress...")
print("="*70)
print(f"Job ID: {job_id}")
print(f"Started monitoring at: {datetime.now().strftime('%H:%M:%S')}")
print("\n💡 This will check every 5 minutes until complete")
print("💡 You can stop and restart this cell anytime\n")
print("="*70)

# Monitor loop
check_count = 0

while True:
    check_count += 1

    # Get job status
    job = client.fine_tuning.jobs.retrieve(job_id)
    status = job.status

    print(f"\n[Check #{check_count}] {datetime.now().strftime('%H:%M:%S')}")
    print(f"Status: {status}")

    if status == "succeeded":
        print("\n" + "🎉"*35)
        print("FINE-TUNING COMPLETE!")
        print("🎉"*35)

        finetuned_model_id = job.fine_tuned_model
        print(f"\n✅ Fine-tuned Model ID: {finetuned_model_id}")

        # Save model ID
        model_info = {
            'model_id': finetuned_model_id,
            'base_model': 'gpt-3.5-turbo',
            'job_id': job_id,
            'completed_at': datetime.now().isoformat(),
            'training_examples': job_info['num_examples'],
            'epochs': job_info['n_epochs']
        }

        model_path = os.path.join(project_dir, 'finetuned_model_info.json')
        with open(model_path, 'w') as f:
            json.dump(model_info, f, indent=2)

        print(f"\n💾 Model info saved to: {model_path}")

        # Also save just the model ID for easy access
        with open(os.path.join(project_dir, 'finetuned_model_id.txt'), 'w') as f:
            f.write(finetuned_model_id)

        print("\n✅ Ready to use in Integration notebook!")
        print(f"📓 Next: Create Notebook 3 for integration")

        break

    elif status == "failed":
        print("\n❌ Fine-tuning FAILED!")
        print(f"Error: {job.error}")
        break

    elif status == "cancelled":
        print("\n⚠️  Fine-tuning was cancelled")
        break

    else:
        # Still running
        if hasattr(job, 'trained_tokens'):
            print(f"Progress: {job.trained_tokens} tokens trained")

        print("⏳ Still training... will check again in 5 minutes")
        print("💤 Sleeping...")

        time.sleep(300)  # Wait 5 minutes

print("\n" + "="*70)
print("Monitoring ended at:", datetime.now().strftime('%H:%M:%S'))

🔄 Monitoring Fine-Tuning Progress...
Job ID: ftjob-B6muR1Rh2BEe5JWIs5W9O3Jv
Started monitoring at: 02:43:01

💡 This will check every 5 minutes until complete
💡 You can stop and restart this cell anytime


[Check #1] 02:43:01
Status: validating_files
Progress: None tokens trained
⏳ Still training... will check again in 5 minutes
💤 Sleeping...

[Check #2] 02:48:02
Status: running
Progress: None tokens trained
⏳ Still training... will check again in 5 minutes
💤 Sleeping...

[Check #3] 02:53:02
Status: running
Progress: None tokens trained
⏳ Still training... will check again in 5 minutes
💤 Sleeping...

[Check #4] 02:58:03
Status: running
Progress: None tokens trained
⏳ Still training... will check again in 5 minutes
💤 Sleeping...

[Check #5] 03:03:03
Status: running
Progress: None tokens trained
⏳ Still training... will check again in 5 minutes
💤 Sleeping...

[Check #6] 03:08:03
Status: running
Progress: None tokens trained
⏳ Still training... will check again in 5 minutes
💤 Sleeping...



In [ ]:
# Cell 9: Test the Fine-Tuned Model (Local Runtime)

from openai import OpenAI
import json
import os

# Local project directory
project_dir = os.path.expanduser('~/FinancialAI')

# Load fine-tuned model ID
with open(os.path.join(project_dir, 'finetuned_model_id.txt'), 'r') as f:
    finetuned_model_id = f.read().strip()

# Create OpenAI client
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

print("🧪 Testing Fine-Tuned Model")
print("="*70)
print(f"Model: {finetuned_model_id}\n")

# Test questions
test_cases = [
    {
        "context": """Apple Inc. Financial Data:

Revenue breakdown for fiscal 2023:
- iPhone: $200.6 billion
- Mac: $29.4 billion
- iPad: $28.3 billion
- Wearables: $39.8 billion
- Services: $85.2 billion

Total net sales: $383.3 billion
Cost of sales: $214.1 billion
Gross profit: $169.2 billion""",
        "question": "What was Apple's gross margin percentage in fiscal 2023?"
    },
    {
        "context": """Microsoft Corporation reported the following for fiscal year 2023:

Segment Revenue:
- Productivity and Business Processes: $69.3 billion
- Intelligent Cloud: $87.9 billion
- More Personal Computing: $54.7 billion

Total Revenue: $211.9 billion
Operating Income: $88.5 billion""",
        "question": "What percentage of Microsoft's total revenue came from Intelligent Cloud?"
    },
    {
        "context": """Tesla Financial Summary:

Q1 2023: Revenue $23.3B, Net Income $2.5B
Q2 2023: Revenue $24.9B, Net Income $2.7B
Q3 2023: Revenue $23.4B, Net Income $1.9B
Q4 2023: Revenue $25.2B, Net Income $2.3B""",
        "question": "What was Tesla's total revenue for 2023?"
    }
]

# Test each case
for i, test in enumerate(test_cases, 1):
    print(f"\n{'='*70}")
    print(f"TEST {i}/{len(test_cases)}")
    print(f"{'='*70}")

    print(f"\n📄 Context:\n{test['context']}")
    print(f"\n❓ Question: {test['question']}")

    # Get answer from fine-tuned model
    response = client.chat.completions.create(
        model=finetuned_model_id,
        messages=[
            {
                "role": "system",
                "content": "You are an expert financial analyst. Answer questions accurately based on the provided financial data."
            },
            {
                "role": "user",
                "content": f"Context:\n{test['context']}\n\nQuestion: {test['question']}"
            }
        ],
        temperature=0.3,
        max_tokens=300
    )

    answer = response.choices[0].message.content

    print(f"\n💡 Fine-Tuned Model Answer:")
    print(f"{answer}")

    print(f"\n✅ Test {i} complete")

print("\n" + "="*70)
print("🎉 ALL TESTS COMPLETE!")
print("="*70)
print("\n✅ Fine-tuned model is working correctly!")
print(f"📓 Model ID: {finetuned_model_id}")
print("\n🚀 Next Step: Create Notebook 3 for full integration")

🧪 Testing Fine-Tuned Model
Model: ft:gpt-4o-2024-08-06:personal:finqa-financial:Chr7KFPi


TEST 1/3

📄 Context:
Apple Inc. Financial Data:
        
Revenue breakdown for fiscal 2023:
- iPhone: $200.6 billion
- Mac: $29.4 billion  
- iPad: $28.3 billion
- Wearables: $39.8 billion
- Services: $85.2 billion

Total net sales: $383.3 billion
Cost of sales: $214.1 billion
Gross profit: $169.2 billion

❓ Question: What was Apple's gross margin percentage in fiscal 2023?

💡 Fine-Tuned Model Answer:
44.1%

✅ Test 1 complete

TEST 2/3

📄 Context:
Microsoft Corporation reported the following for fiscal year 2023:

Segment Revenue:
- Productivity and Business Processes: $69.3 billion
- Intelligent Cloud: $87.9 billion  
- More Personal Computing: $54.7 billion

Total Revenue: $211.9 billion
Operating Income: $88.5 billion

❓ Question: What percentage of Microsoft's total revenue came from Intelligent Cloud?

💡 Fine-Tuned Model Answer:
41%

✅ Test 2 complete

TEST 3/3

📄 Context:
Tesla Financial Su

In [ ]:
# Cell 10: Estimate Fine-Tuning Cost (Local Runtime)

import json
import os

# Local project directory
project_dir = os.path.expanduser('~/FinancialAI')

# Load job info
with open(os.path.join(project_dir, 'finetuning_job_info.json'), 'r') as f:
    job_info = json.load(f)

num_examples = job_info['num_examples']
n_epochs = job_info['n_epochs']

# Estimate tokens per example (average for FinQA)
avg_tokens_per_example = 800  # Context + Question + Answer

# Total training tokens
total_tokens = num_examples * n_epochs * avg_tokens_per_example

# OpenAI pricing (as of Nov 2024)
training_cost_per_1k_tokens = 0.008  # $0.008 per 1K tokens

# Calculate cost
estimated_cost = (total_tokens / 1000) * training_cost_per_1k_tokens

print("💰 FINE-TUNING COST ESTIMATION")
print("="*70)
print(f"\n📊 Training Details:")
print(f"   Examples: {num_examples:,}")
print(f"   Epochs: {n_epochs}")
print(f"   Avg tokens/example: {avg_tokens_per_example}")
print(f"\n📊 Total Training Tokens: {total_tokens:,}")
print(f"\n💵 Estimated Cost: ${estimated_cost:.2f}")

print(f"\n💡 Additional costs:")
print(f"   • Inference: ~$0.002 per query")
print(f"   • Total for 1000 queries: ~$2.00")

print(f"\n💰 Total Project Cost Estimate: ${estimated_cost + 2:.2f}")

print("\n" + "="*70)

💰 FINE-TUNING COST ESTIMATION

📊 Training Details:
   Examples: 984
   Epochs: 3
   Avg tokens/example: 800

📊 Total Training Tokens: 2,361,600

💵 Estimated Cost: $18.89

💡 Additional costs:
   • Inference: ~$0.002 per query
   • Total for 1000 queries: ~$2.00

💰 Total Project Cost Estimate: $20.89



In [ ]:
# Run this to get your model ID

from openai import OpenAI
import os
import json

project_dir = os.path.expanduser('~/FinancialAI')
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Load job info
with open(os.path.join(project_dir, 'finetuning_job_info.json'), 'r') as f:
    job_info = json.load(f)

job_id = job_info['job_id']

# Get job details
job = client.fine_tuning.jobs.retrieve(job_id)

print("🎉 FINE-TUNING COMPLETE!")
print("="*70)
print(f"Status: {job.status}")
print(f"Model ID: {job.fine_tuned_model}")
print("="*70)

# Save model ID
finetuned_model_id = job.fine_tuned_model

with open(os.path.join(project_dir, 'finetuned_model_id.txt'), 'w') as f:
    f.write(finetuned_model_id)

print(f"\n✅ Model ID saved to: {project_dir}/finetuned_model_id.txt")
print(f"\n📋 Your Fine-Tuned Model: {finetuned_model_id}")

🎉 FINE-TUNING COMPLETE!
Status: succeeded
Model ID: ft:gpt-4o-2024-08-06:personal:finqa-financial:Chr7KFPi

✅ Model ID saved to: /root/FinancialAI/finetuned_model_id.txt

📋 Your Fine-Tuned Model: ft:gpt-4o-2024-08-06:personal:finqa-financial:Chr7KFPi


In [ ]:
# Cell: Test Fine-Tuned GPT-4o Model

from openai import OpenAI
import os

project_dir = os.path.expanduser('~/FinancialAI')
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Load model ID
with open(os.path.join(project_dir, 'finetuned_model_id.txt'), 'r') as f:
    finetuned_model_id = f.read().strip()

print("🧪 Testing Fine-Tuned Model")
print("="*70)
print(f"Model: {finetuned_model_id}\n")

# Test with a financial question
test_context = """
Apple Inc. Financial Data (Fiscal 2023):

Revenue by Segment:
- iPhone: $200.6 billion
- Mac: $29.4 billion
- iPad: $28.3 billion
- Wearables, Home, Accessories: $39.8 billion
- Services: $85.2 billion

Total Revenue: $383.3 billion
Cost of Revenue: $214.1 billion
Gross Profit: $169.2 billion
Operating Income: $114.3 billion
Net Income: $97.0 billion
"""

test_question = "What was Apple's gross profit margin in fiscal 2023?"

print("📄 Context:")
print(test_context)
print(f"\n❓ Question: {test_question}")

# Call fine-tuned model
response = client.chat.completions.create(
    model=finetuned_model_id,  # ← Your fine-tuned model
    messages=[
        {
            "role": "system",
            "content": "You are an expert financial analyst. Answer questions accurately based on the provided financial data."
        },
        {
            "role": "user",
            "content": f"Context:\n{test_context}\n\nQuestion: {test_question}"
        }
    ],
    temperature=0.3,
    max_tokens=300
)

answer = response.choices[0].message.content

print("\n💡 Fine-Tuned Model Answer:")
print("="*70)
print(answer)
print("="*70)

print("\n✅ Fine-tuned model is working!")

🧪 Testing Fine-Tuned Model
Model: ft:gpt-4o-2024-08-06:personal:finqa-financial:Chr7KFPi

📄 Context:

Apple Inc. Financial Data (Fiscal 2023):

Revenue by Segment:
- iPhone: $200.6 billion
- Mac: $29.4 billion
- iPad: $28.3 billion
- Wearables, Home, Accessories: $39.8 billion
- Services: $85.2 billion

Total Revenue: $383.3 billion
Cost of Revenue: $214.1 billion
Gross Profit: $169.2 billion
Operating Income: $114.3 billion
Net Income: $97.0 billion


❓ Question: What was Apple's gross profit margin in fiscal 2023?

💡 Fine-Tuned Model Answer:
44.1%

✅ Fine-tuned model is working!


In [ ]:
# Cell: Compare Fine-Tuned vs Baseline

from openai import OpenAI
import os

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
project_dir = os.path.expanduser('~/FinancialAI')

# Load fine-tuned model ID
with open(os.path.join(project_dir, 'finetuned_model_id.txt'), 'r') as f:
    finetuned_model_id = f.read().strip()

# Test questions
test_cases = [
    {
        "context": "Company X: Revenue $100M (2022), $120M (2023). Operating Income $20M (2022), $28M (2023).",
        "question": "What was the operating margin improvement?"
    },
    {
        "context": "Tesla Q3 2023: Total revenue $23.4B, Automotive revenue $19.6B, Energy revenue $1.6B, Services $2.2B.",
        "question": "What percentage of revenue came from automotive sales?"
    }
]

print("🔬 COMPARISON: Fine-Tuned vs Baseline")
print("="*70)

for i, test in enumerate(test_cases, 1):
    print(f"\n{'='*70}")
    print(f"TEST CASE {i}")
    print(f"{'='*70}")
    print(f"\n📄 Context: {test['context']}")
    print(f"\n❓ Question: {test['question']}")

    prompt = f"Context:\n{test['context']}\n\nQuestion: {test['question']}"

    # Baseline GPT-3.5
    print("\n🔵 BASELINE (GPT-3.5-turbo):")
    print("-"*70)
    baseline_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a financial analyst."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    print(baseline_response.choices[0].message.content)

    # Fine-tuned GPT-4o
    print("\n🟢 FINE-TUNED (GPT-4o):")
    print("-"*70)
    finetuned_response = client.chat.completions.create(
        model=finetuned_model_id,
        messages=[
            {"role": "system", "content": "You are a financial analyst."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    print(finetuned_response.choices[0].message.content)

print("\n" + "="*70)
print("✅ Comparison complete!")

🔬 COMPARISON: Fine-Tuned vs Baseline

TEST CASE 1

📄 Context: Company X: Revenue $100M (2022), $120M (2023). Operating Income $20M (2022), $28M (2023).

❓ Question: What was the operating margin improvement?

🔵 BASELINE (GPT-3.5-turbo):
----------------------------------------------------------------------
To calculate the operating margin improvement, we first need to calculate the operating margin for each year.

Operating Margin = Operating Income / Revenue

For 2022:
Operating Margin 2022 = $20M / $100M = 0.20 or 20%

For 2023:
Operating Margin 2023 = $28M / $120M = 0.2333 or 23.33%

Now, we can calculate the operating margin improvement by comparing the operating margin in 2023 to that in 2022:

Operating Margin Improvement = Operating Margin 2023 - Operating Margin 2022
Operating Margin Improvement = 23.33% - 20% = 3.33%

Therefore, the operating margin improvement from 2022 to 2023 for Company X is 3.33%.

🟢 FINE-TUNED (GPT-4o):
--------------------------------------------------